# Bllossom-8B를 이용한 한국어 LLM 튜토리얼

> [유튜브 빵형의 개발도상국](https://www.youtube.com/@bbanghyong)

https://huggingface.co/MLP-KTLim/llama3-Bllossom

## 01. 활용할 package 설정
 - GPU사용하기: colab에서 런타임 --> 런타임유형변경 --> T4 선택
 - 패키지설치: 아래 pip를 이용해 Transformers, accelerate 설치
 - 런타임재시작: 런타임 --> 세션다시시작  (accelerate설치 시 런타임 다시시작하셔야됩니다!)

In [24]:
!pip install -q transformers==4.40.0 accelerate gradio

In [25]:
!pip install torch

## 02. 모델준비

In [26]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'MLP-KTLim/llama3-Bllossom'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()

NameError: name '_C' is not defined

In [ ]:
tokenizer.eos_token_id

NameError: name 'tokenizer' is not defined

In [ ]:
model_id

'MLP-KTLim/llama3-Bllossom'

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [ ]:
model.config

LlamaConfig {
  "_name_or_path": "MLP-KTLim/llama3-Bllossom",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.40.0",
  "use_cache": false,
  "vocab_size": 128256
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 03. 추론

In [ ]:
PROMPT = '''당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.'''
instruction = "다음 제목의 논문을 요약해줘 'Optimizing Language Augmentation for Multilingual Large Language Models: A Case Study on Korean'"

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    repetition_penalty=1.1
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


논문 요약:

논문 "Optimizing Language Augmentation for Multilingual Large Language Models: A Case Study on Korean"에서는 다중 언어 대규모 언어 모델(Large Language Models, LLMs)의 성능을 향상시키기 위해 언어 증강(language augmentation)을 최적화하는 방법을 탐구합니다. 특히 이 연구는 한국어를 대상으로 한 사례 연구를 통해 이러한 문제를 해결하기 위한 접근법을 제안합니다.

### 주요 내용

1. **언어 증강의 중요성**: 자연어 처리(NLP) 작업에서 언어 모델의 학습 데이터 양과 품질이 매우 중요합니다. 그러나 특정 언어에 대한 데이터가 부족하거나 불균형일 경우, 모델의 일반화 능력과 성능이 저하될 수 있습니다. 이를 해결하기 위해 언어 증강 기법이 사용됩니다.

2. **현재 상태와 한계**: 기존의 언어 증강 기법들은 주로 단어-level 또는 문장-level에서 이루어지며, 다중 언어 환경에서의 적용에는 한계가 있습니다. 또한, 각 언어의 문법적 구조와 문화적 특성이 고려되지 않아 성능 차이를 초래할 수 있습니다


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델과 토크나이저 로드
model_id = 'MLP-KTLim/llama3-Bllossom'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # bfloat16을 사용하여 메모리 최적화
    device_map="auto"            # GPU 사용이 가능할 경우 자동 할당
)
model.eval()  # 모델을 평가 모드로 설정

def generate_text(prompt, max_length=100, temperature=0.7, top_p=0.9):
    # 입력 프롬프트를 토크나이징
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # 모델을 통해 문장 생성
    with torch.no_grad():  # 평가 모드에서는 그래디언트 계산 비활성화
        outputs = model.generate(
            input_ids,
            max_length=max_length,   # 생성할 최대 문장의 길이
            do_sample=True,          # 샘플링 방식 사용
            temperature=temperature, # 생성의 무작위성 조절
            top_p=top_p,             # Top-p 샘플링을 통해 확률 상위 토큰들 중 선택
            repetition_penalty=1.1,  # 반복 방지
            eos_token_id=tokenizer.eos_token_id  # 문장이 끝나는 지점을 정의
        )

    # 생성된 토큰을 텍스트로 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 예시 프롬프트로 문장 생성
prompt = "한국의 인공지능 발전을 위해 필요한 요소는"
generated_output = generate_text(prompt)

# 생성된 텍스트 출력
print("생성된 텍스트:", generated_output)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


생성된 텍스트: 한국의 인공지능 발전을 위해 필요한 요소는 무엇일까요? 먼저, 정부와 민간 기업이 함께 협력하여 인공지능 연구개발에 대한 투자와 지원이 필요합니다. 또한, 데이터의 수집과 분석이 중요한데, 이를 위해서는 개인정보 보호와 관련된 법적 구조가 마련되어야 합니다. 그리고 교육과 훈련도 중요합니다. 인공지능 기술이 다양하게 적용되기 위해서는 기존


In [ ]:
PROMPT = '''당신은 유용한 AI 어시스턴트입니다. 제시해주는 원관념(핵심역량)을 통해서 새로운 보조관념과 그에 따른 비유문장 생성해야돼 '''
instruction = "'나의 핵심역량(원관념)은 전문성이고 보조관념(비행기 기장)이랑 비유문장 그리고 이유도 설명해봐 '"

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
    repetition_penalty=1.1
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


### 원관념: 전문성

**보조관념 1: 지식**
- **비유문장:** "전문성이란 하나의 큰 책방에서 수많은 책을 잘 알고 있는 도서관 관리자와 같다. 다양한 분야의 지식을 소화하고, 언제든지 필요한 정보를 빠르게 찾아내어 문제를 해결할 수 있어야 한다."
- **이유:** 지식은 전문성을 구성하는 중요한 요소로, 특정 분야에서 깊이 있고 넓게 이해된 정보는 전문가로서의 신뢰성을 높인다.

**보조관념 2: 실무 경험**
- **비유문장:** "전문성이란 강한 물건을 다루는 기술을 가진 조각사와 같다. 시간이 지나면서 점점 더 정교하고 복잡한 작품을 만들어낼 수 있게 되어야 한다."
- **이유:** 실제로 많은 상황에서 적용해보며 쌓은 실무 경험은 전문성을 더욱 풍부하게 하고, 예상치 못한 상황에서도 효과적으로 대처할 수 있는 능력을 키준다.

**보조관념 3: 분석 및 문제 해결 능력**
- **


In [ ]:
import torch

torch.cuda.empty_cache()
#gpu 안쓰는 메모리 정리?하는 코드

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델과 토크나이저 로드
model_id = 'MLP-KTLim/llama3-Bllossom'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # bfloat16을 사용하여 메모리 최적화
    device_map="auto"            # GPU 사용이 가능할 경우 자동 할당
)
model.eval()  # 모델을 평가 모드로 설정

# 시스템 프롬프트와 사용자의 요청을 병합한 프롬프트
PROMPT = '''당신은 유용한 AI 어시스턴트입니다. 제시해주는 원관념(핵심역량)을 통해서 새로운 보조관념과 그에 따른 비유문장 생성해야돼 '''
instruction = "나의 핵심역량(원관념)은 전문성이고 보조관념(비행기 기장)이랑 비유문장 그리고 이유도 설명해봐 "

# 프롬프트 병합
full_prompt = f"{PROMPT}\n{instruction}"

def generate_text(prompt, max_length=256, temperature=0.7, top_p=0.9):
    # 입력 프롬프트를 토크나이징
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # 모델을 통해 문장 생성
    with torch.no_grad():  # 평가 모드에서는 그래디언트 계산 비활성화
        outputs = model.generate(
            input_ids,
            max_length=max_length,   # 생성할 최대 문장의 길이
            do_sample=True,          # 샘플링 방식 사용
            temperature=temperature, # 생성의 무작위성 조절
            top_p=top_p,             # Top-p 샘플링을 통해 확률 상위 토큰들 중 선택
            repetition_penalty=1.1,  # 반복 방지
            eos_token_id=tokenizer.eos_token_id  # 문장이 끝나는 지점을 정의
        )

    # 생성된 토큰을 텍스트로 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 병합된 프롬프트로 문장 생성
generated_output = generate_text(full_prompt)

# 생성된 텍스트 출력
print("생성된 텍스트:", generated_output)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 134221824 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


생성된 텍스트: 당신은 유용한 AI 어시스턴트입니다. 제시해주는 원관념(핵심역량)을 통해서 새로운 보조관념과 그에 따른 비유문장 생성해야돼 
나의 핵심역량(원관념)은 전문성이고 보조관념(비행기 기장)이랑 비유문장 그리고 이유도 설명해봐 





---

**원관념: 전문성**
**보조관념: 비행기 기장**
**비유문장:** "전문성이란 비행기를 안전하게 지휘하는 기장을 닮아, 복잡한 상황에서도 정확하고 효율적으로 문제를 해결할 수 있는 능력을 의미한다."

**이유:**
비행기 기장은 안전한 착륙을 위해 다양한 요소를 고려하여 조종을 해야 한다. 마찬가지로 전문가는 자신의 분야에서 최신 정보와 경험을 바탕으로 문제를 해결하며, 이를 통해 더 나은 결과를 도출한다. 또한, 비행기 기장은 항상 주의를 기울여야 하며, 실수를 용납하지 않는 철학을 가지고 있다. 전문가도 자신의 업무를 철


In [ ]:
# 모델과 토크나이저 로드
model_id = 'MLP-KTLim/llama3-Bllossom'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # bfloat16을 사용하여 메모리 최적화
    device_map="auto"            # GPU 사용이 가능할 경우 자동 할당
)
model.eval()  # 모델을 평가 모드로 설정

# 시스템 프롬프트와 사용자의 요청을 병합한 프롬프트
PROMPT1 = '''당신은 유용한 AI 어시스턴트입니다. 제시해주는 원관념(핵심역량)을 통해서 새로운 보조관념과 그에 따른 비유문장 생성해야돼 '''
instruction1 = "나의 핵심역량(원관념)은 전문성, 협응력, 주도성이고 각각에 맞는 보조관념이랑 비유문장 그리고 이유도 설명해봐 "

full_prompt = f"{PROMPT1}\n{instruction1}"

def generate_text(prompt, max_length=512, temperature=0.7, top_p=0.9):
    # 입력 프롬프트를 토크나이징
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # 모델을 통해 문장 생성
    with torch.no_grad():  # 평가 모드에서는 그래디언트 계산 비활성화
        outputs = model.generate(
            input_ids,
            max_length=max_length,   # 생성할 최대 문장의 길이
            do_sample=True,          # 샘플링 방식 사용
            temperature=temperature, # 생성의 무작위성 조절
            top_p=top_p,             # Top-p 샘플링을 통해 확률 상위 토큰들 중 선택
            repetition_penalty=1.1,  # 반복 방지
            eos_token_id=tokenizer.eos_token_id  # 문장이 끝나는 지점을 정의
        )

    # 생성된 토큰을 텍스트로 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 병합된 프롬프트로 문장 생성
generated_output = generate_text(full_prompt)

# 생성된 텍스트 출력
print("생성된 텍스트:", generated_output)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


생성된 텍스트: 당신은 유용한 AI 어시스턴트입니다. 제시해주는 원관념(핵심역량)을 통해서 새로운 보조관념과 그에 따른 비유문장 생성해야돼 
나의 핵심역량(원관념)은 전문성, 협응력, 주도성이고 각각에 맞는 보조관념이랑 비유문장 그리고 이유도 설명해봐 1차원적인 답변보다는 다양한 측면에서 접근해보세요. 

**전문성**
- **보조관념**: 지식, 기술
- **비유문장**: "어떤 사람처럼 전문성을 가지는 것은 강력한 무기가 되며, 이를 통해 문제를 해결하고 목표를 달성하는 능력을 갖춘다."
- **이유**: 전문성은 특정 분야에서 깊이 있는 지식과 기술을 의미한다. 이는 전문가로서의 역할을 수행하는데 필수적이며, 다른 사람들이 어려워하는 문제를 해결하고, 새로운 아이디어와 솔루션을 제시할 수 있게 해준다.

**협응력**
- **보조관념**: 소통, 공감
- **비유문장**: "협응력은 서로 다른 언어로 말하더라도 이해할 수 있는 특별한 통역사역이다. 이는 팀워크와 조화를 이루는 중요한 요소로, 서로를 존중하고 지원하며 성공적인 결과를 도모하는 데 기여한다."
- **이유**: 협응력은 다른 사람들과 효과적으로 소통하고, 그들의 의견을 이해하고 공감하는 능력을 의미한다. 이는 팀워크를 강화하고, 갈등을 줄이고, 더 나은 결정을 내리는 데 중요하다. 또한, 다양한 관점을 고려하여 더 창의적이고 혁신적인 접근 방식을 모색할 수 있게 한다.

**주도성**
- **보조관념**: 리더십,_INITIATIVE_
- **비유문장**: "주도성이 있는 사람은 불확실한 상황에서도 선두에 서서 길을 열고, 동료들을 이끌어 항상 새로운 도전에 나선다. 이는 팀 전체가 성장하고 발전할 수 있도록 하는 강력한 동력이 된다."
- **이유**: 주도성은 자신의 아이디어를 추진하고, 새로운 도전을


In [ ]:
#prompt 수정 (창의성 추가)

In [ ]:
# 모델과 토크나이저 로드
model_id = 'MLP-KTLim/llama3-Bllossom'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # bfloat16을 사용하여 메모리 최적화
    device_map="auto"            # GPU 사용이 가능할 경우 자동 할당
)
model.eval()  # 모델을 평가 모드로 설정

# 시스템 프롬프트와 사용자의 요청을 병합한 프롬프트
PROMPT2 = '''당신은 유용한 AI 어시스턴트입니다. 제시해주는 원관념(핵심역량)을 통해서 새로운 '창의성을 가진' 보조관념과 그에 따른 비유문장 생성해야돼 '''
instruction1 = "나의 핵심역량(원관념)은 전문성, 협응력, 주도성이고 각각에 맞는 보조관념이랑 비유문장 그리고 이유도 설명해봐 "

full_prompt = f"{PROMPT1}\n{instruction1}"

def generate_text(prompt, max_length=512, temperature=0.7, top_p=0.9):
    # 입력 프롬프트를 토크나이징
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # 모델을 통해 문장 생성
    with torch.no_grad():  # 평가 모드에서는 그래디언트 계산 비활성화
        outputs = model.generate(
            input_ids,
            max_length=max_length,   # 생성할 최대 문장의 길이
            do_sample=True,          # 샘플링 방식 사용
            temperature=temperature, # 생성의 무작위성 조절
            top_p=top_p,             # Top-p 샘플링을 통해 확률 상위 토큰들 중 선택
            repetition_penalty=1.1,  # 반복 방지
            eos_token_id=tokenizer.eos_token_id  # 문장이 끝나는 지점을 정의
        )

    # 생성된 토큰을 텍스트로 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 병합된 프롬프트로 문장 생성
generated_output = generate_text(full_prompt)

# 생성된 텍스트 출력
print("생성된 텍스트:", generated_output)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 134221824 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


생성된 텍스트: 당신은 유용한 AI 어시스턴트입니다. 제시해주는 원관념(핵심역량)을 통해서 새로운 보조관념과 그에 따른 비유문장 생성해야돼 
나의 핵심역량(원관념)은 전문성, 협응력, 주도성이고 각각에 맞는 보조관념이랑 비유문장 그리고 이유도 설명해봐  요약하면 다음과 같습니다.

1. **전문성**
   - 보조관념: 깊이 있는 지식
   - 비유문장: "그의 전문성이 빛나는 별처럼 다른 사람들에게 희망을 주는 길이 되듯이, 우리 팀에서도 그의 역할이 매우 중요합니다."
   - 이유: 전문성을 가진 사람이 팀 내에서 리더로서 중요한 역할을 할 수 있으며, 그들의 지식과 경험은 팀의 성장을 돕습니다. 비유문장은 이러한 전문성을 강조하고, 팀에서의 중요성을 시각적으로 표현합니다.

2. **협응력**
   - 보조관념: 조화로운 상호작용
   - 비유문장: "팀워크를 위한 유명한 멜로디가 들려오는 것처럼, 모두가 서로의 장점을 존중하며 조화를 이루는 모습이 참으로 아름답습니다."
   - 이유: 협응력이 높은 팀은 서로의 강점을 최대한 활용할 수 있으며, 이를 통해 더 나은 결과를 얻을 수 있습니다. 비유문장은 협업의 중요성을 강조하고, 조화로운 상호작용을 시각적으로 표현합니다.

3. **주도성**
   - 보조관념: 목표 달성
   - 비유문장: "목표를 향해 가는 열차처럼, 그가 앞서 나서는 것은 우리 모두에게 도움이 됩니다. 함께 끌어올리는 열차로, 더욱 큰 성공을 향해 달리겠습니다."
   - 이유: 주도성이 있는 사람은 목표를 설정하고 이를 달성하기 위해 노력하는 역할을 합니다. 비유문장은 이러한 주도성을 강조하고, 목표 달성을 위한 팀의 협력을 시각적으로 표현합니다.

이러한 원관념과 보조관념, 비유문장을 통해 당신은 더 나은 방식으로 내가 제공하는 정보를 이해하고, 이를 바탕


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'MLP-KTLim/llama3-Bllossom'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model.eval()

NameError: name '_C' is not defined

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import bitsandbytes as bnb  # 8비트 양자화를 위한 라이브러리

# CUDA 사용 가능 여부 확인
if torch.cuda.is_available():
    print("CUDA is available. GPU를 사용할 수 있습니다.")
else:
    print("CUDA is not available. CPU만 사용할 수 있습니다.")

# 모델과 토크나이저 로드
model_id = 'MLP-KTLim/llama3-Bllossom'

tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델을 8비트 양자화하여 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,             # 8비트 양자화 활성화
    device_map="auto",             # GPU 자동 할당
    torch_dtype=torch.float16,     # Mixed Precision (bfloat16 -> float16)
    quantization_config=bnb.QuantizationConfig(bits=8)  # 8비트 양자화 설정
)

# Gradient Checkpointing 활성화
model.gradient_checkpointing_enable()

model.eval()  # 모델을 평가 모드로 설정

# 키워드를 기반으로 하는 새로운 프롬프트와 지시사항
PROMPT = '''넌 굉장히 창의적인 모델이야'''

# 사용자 요청에 따라 instruction을 구성
instruction = "나의 핵심역량(원관념)은 전문성, 협응력, 주도성, 기획, 진행이고 각각에 맞는 비유문장과 설명을 작성해주세요."

# 프롬프트 병합
full_prompt = f"{PROMPT}\n{instruction}"

def generate_text(prompt, max_length=1024, temperature=1.2, top_p=0.9):
    # 입력 프롬프트를 토크나이징
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # 모델을 통해 문장 생성
    with torch.no_grad():  # 평가 모드에서는 그래디언트 계산 비활성화
        outputs = model.generate(
            input_ids,
            max_length=max_length,   # 생성할 최대 문장의 길이
            do_sample=True,          # 샘플링 방식 사용
            temperature=temperature, # 생성의 무작위성 조절
            top_p=top_p,             # Top-p 샘플링을 통해 확률 상위 토큰들 중 선택
            repetition_penalty=1.1,  # 반복 방지
            eos_token_id=tokenizer.eos_token_id  # 문장이 끝나는 지점을 정의
        )

    # 생성된 토큰을 텍스트로 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 병합된 프롬프트로 문장 생성
generated_output = generate_text(full_prompt)

# 생성된 텍스트 출력
print("생성된 텍스트:", generated_output)


NameError: name '_C' is not defined

## 04. 챗봇 인터페이스

In [ ]:
import gradio as gr

# PROMPT = '''당신은 20대 초반의 여성이고 이름은 '빵순이'입니다. 귀여운 말투로 메신저로 대화하듯이 간결하고 정확하게 대답해야 합니다.'''
PROMPT = '''당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.'''

def generate_response(input_text, chat_history):
    messages = [{"role": "system", "content": f"{PROMPT}"}]

    for message in chat_history:
        messages.append({"role": "user", "content": message[0]})
        messages.append({"role": "assistant", "content": message[1]})

    messages.append({"role": "user", "content": input_text})

    input_ids = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.1
    )

    response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    chat_history.append((input_text, response))

    return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=600)
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    msg.submit(generate_response, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:222: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://914511682ae648c60a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
